In [ ]:
#
#    IMPORTS
# 

#    Import some standard python modules
import os, json, time
#
#    The module pyproj is a standard module that can be installed using pip or your other favorite
#    installation tool. This module provides tools to convert between different geodesic coordinate systems
#    and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
#
#    The 'wildfire' module is a user module. This module is available from the course website. The module
#    includes one object, a Reader, that can be used to read the GeoJSON files associated with the
#    wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that
#    contains a small number of California wildfires extracted from the main wildfire dataset.
#    
from wildfire import Reader 
#
#    There is a GeoJSON reader that you might try if you wanted to read the data. It has its own quirks.
#    There will be an example below that reads the sample file "Wildfire_short_sample.json"
#    
import geojson

import pandas as pd

Here, we use the 'wildfire' module to create an instance of the Reader class. We open the data file and print out the headers in order to view the fields present in the data.

In the following cells we provide small code snippets that do the following:

Create a wildfire Reader() object and use it to open the sample data file. Once, opened, we have access to the header information so we print that to show the structure of that data.
Use the Reader() object and the next() method to read the set of wildfire features. 
Access the geometry of the feature to get the 'ring' boundary of that specific fire - which is a list of geodetic coordinates.

Another note regarding terminology. In the GeoJSON standard, something that is to be represented geographically is generically called a 'feature'. In the case of the wildfire dataset every 'feature' is a wildfire. These terms are used somewhat interchangably below.

In [ ]:
reader = Reader.Reader()
reader.open("GeoJSON Exports/USGS_Wildland_Fire_Combined_Dataset.json")

In [ ]:
header_dict = reader.header()
header_keys = list(header_dict.keys())
print("The header has the following keys:")
print(header_keys)
print()
print("Header Dictionary")
print(json.dumps(header_dict,indent=4))

In [ ]:
feature_list = list()
feature_count = 0
# A rewind() on the reader object makes sure we're at the start of the feature list
# This way, we can execute this cell multiple times and get the same result 
reader.rewind()
# Now, read through each of the features, saving them as dictionaries into a list
feature = reader.next()
while feature:
    feature_list.append(feature)
    feature_count += 1
    # if we're loading a lot of features, print progress
    if (feature_count % 10000) == 0:
        print(f"Loaded {feature_count} features")
    # loaded the max we're allowed then break
    feature = reader.next()
#
#    Print the number of items (features) we think we loaded
print(f"Loaded a total of {feature_count} features")
#
#    Just a validation check - did all the items we loaded get into the list?
print(f"Variable 'feature_list' contains {len(feature_list)} features")

The following helper functions will allow us to calculate the shortest and average distance from the fire to te city.

The first bit of code finds the point on the perimiter with the shortest distance to the city (place) and returns the distance as well as the lat,lon of the perimeter point.

The second bit of code calculates the average distance of all perimeter points to the city (place) and returns that average as the distance. This is not quite what the centroid would be, but it is probably fairly close.

These are two reasonable ways to think about possible distance to a fire. But both require computing distance to a whole set of points on the perimeter of a fire.

In [ ]:
# define helper functions for the tasks
# 1. 
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

# 2.
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point


# 3.
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
#
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average

In [ ]:
city_latlon = [38.052147, -122.153893]

final_wildfire_data = []

for wildfire_feature in feature_list:
    
    fire_year = wildfire_feature['attributes']['Fire_Year']
    fire_name = wildfire_feature['attributes']['Listed_Fire_Names'].split(',')[0]
    
    print(f"Analyzing wildfire {fire_name} in the year {fire_year}.")
    
    if fire_year >= 1963:
        
        # Get the geometry for the feature we pulled from the feature_list
        wildfire_geometry = wildfire_feature['geometry']
        
        # Get all the coordinates for the fire boundary
        if list(wildfire_feature['geometry'].keys())[0] == 'curveRings':
            continue;

        ring_data = wildfire_feature['geometry']['rings'][0]
        
        # Compute the average distance between the fire boundary and the city of interest
        distance = average_distance_from_place_to_fire_perimeter(city_latlon,ring_data)
        
        print((f"Wildfire found to have an average distance of {distance:1.2f} miles from Benicia, California"))
        
        if distance <= 1250:
            wildfire_feature['attributes']['Distance'] = distance
            final_wildfire_data.append(wildfire_feature['attributes'])
    

In [ ]:
# Specify the file path where you want to save the JSON data
file_path = "wildfire_data_bencia_ca.json"

# Open the file in write mode and save the data as JSON
with open(file_path, 'w') as json_file:
    json.dump(final_wildfire_data, json_file)

We print one of the features just to explore the dataset's attributes.

In [ ]:
final_wildfire_data[0]

Creating a Smoke Estimate

Having reviewed the metadata and looked over what the features in the dataset look like, I have concluded that a reasonable smoke estimate could be derived from the following factors:
1. The GIS calculated hectares of the fire polygon calculated by using the Calculate Geometry tool in ArcGIS Pro - GIS Hectares
2. The average distance from the fire boundary and city of interest ie Benicia, CA - Distance

It is a reasonable assumption that a fire spread over a larger area will generate more smoke. Similarly, a fire burning further away from the city will have less impact on the city in terms of smoke. Based on these two assumptions, the formula that I will be using for the smoke estimate is:

smoke estimate = (GIS_Hectares * 0.00386)/Distance 

Note: The GIS_Hectares value is multiplied by 0.00386 in order to convert the unit to miles^2.


In [ ]:
wildfire_data_with_estimate = final_wildfire_data

for wf_feature in wildfire_data_with_estimate:
    try:
        area_of_fire = wf_feature['GIS_Hectares']*0.00386
        dist_of_fire = wf_feature['Distance']
    except:
        print("There is a missing attribute")
    smoke_estimate = area_of_fire/dist_of_fire
    wf_feature['Smoke_Estimate'] = smoke_estimate

After calculating the smoke estimate, we create a dataframe to store the necessary attributes for future analysis.

In [ ]:
wildfire_est_list = []
for wf_feature in wildfire_data_with_estimate:
    feature = {}
    feature['USGS_Assigned_ID'] = wf_feature['USGS_Assigned_ID']
    feature['Assigned_Fire_Type'] = wf_feature['Assigned_Fire_Type']
    feature['Fire_Year'] = wf_feature['Fire_Year']
    feature['GIS_Hectares'] = wf_feature['GIS_Hectares']
    feature['GIS_Acres'] = wf_feature['GIS_Acres']
    feature['Listed_Fire_Names'] = wf_feature['Listed_Fire_Names']
    feature['Distance'] = wf_feature['Distance']
    feature['Smoke_Estimate'] = wf_feature['Smoke_Estimate']
    wildfire_est_list.append(feature)
# convert to a dataframe
wildfire_df = pd.DataFrame(wildfire_est_list)
wildfire_df.head()

Next, we scale the smoke estimates using the minmax scaling technique in order to make the comparable to AQI for the next part of our analysis. 

In [ ]:
original_min = wildfire_df['Smoke_Estimate'].min()
original_max = wildfire_df['Smoke_Estimate'].max()
new_max = 500
new_min = 0
wildfire_df['Smoke_Estimate_Scaled'] = (wildfire_df['Smoke_Estimate'] - original_min) / (original_max - original_min) * (new_max - new_min) + new_min

In [ ]:
wildfire_df['Smoke_Estimate_Scaled'].describe()

In [ ]:
temp = wildfire_df

We remove all the wildfires with a scaled smoke estimate lower than 10 as their impact is very low. The only way to get such a low estimate is if the fire was very small and far away from the city resulting in negligible impact.

In [ ]:
wildfire_df = wildfire_df[wildfire_df['Smoke_Estimate_Scaled'] > 10]

In [ ]:
wildfire_df_grouped = wildfire_df.groupby("Fire_Year")["Smoke_Estimate_Scaled"].mean().reset_index()

In [ ]:
import matplotlib.pyplot as plt
wildfire_df_grouped['Fire_Year'] = pd.to_datetime(wildfire_df_grouped['Fire_Year'], format='%Y')
wildfire_df_grouped.set_index('Fire_Year', inplace=True)
plt.figure(figsize=(12, 6))
plt.plot(wildfire_df_grouped['Smoke_Estimate_Scaled'])
plt.title('Smoke Estimate Over Time')
plt.xlabel('Year')
plt.ylabel('Smoke Estimate')
plt.show()

Autoregressive integrated moving average (ARIMA) models predict future values based on past values. ARIMA makes use of lagged moving averages to smooth time series data. They are widely used in technical analysis to forecast future values. ARIMA is widely used in use cases where the data does not exhibit any seasonality or trends which is why I thought that this model would be appropriate in this case.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

alpha = 0.2
model = ARIMA(wildfire_df_grouped, order=(5,1,0))
model_fit = model.fit()

forecast = model_fit.forecast(25)

In [161]:
forecast_df = pd.DataFrame(forecast)

In [ ]:
wildfire_df['Distance'].describe()

This graph is a histogram showing the number of fires occurring every 50 mile distance from Benicia, California up to the max specified distance. The x-axis represents the distance in miles from the assigned city. The distances are divided into 50-mile intervals. Each interval is represented by a bar on the histogram. The y-axis represents the count or frequency of fires that occurred within each 50-mile distance interval. It shows how many fires were recorded in each specific range. Each bar in the histogram represents one of the 50-mile distance intervals. The height of each bar corresponds to the number of fires that occurred in that distance range.

In [ ]:
bin_edges = list(range(0, 1250, 50))  # From 0 to 600 miles with 50-mile intervals

# Create the histogram
plt.hist(wildfire_df['Distance'], bins=bin_edges, edgecolor='k')

# Set plot labels and title
plt.xlabel('Distance (miles)')
plt.ylabel('Number of Fires')
plt.title('Histogram of Fires by Distance')

# Show the plot
plt.show()

This graph is a time series graph of total acres burned per year for the fires occurring in the specified distance from Benicia, California.  The x-axis represents the years during which the fires occurred. Each year is marked along the x-axis, allowing viewers to follow the progression of time. The y-axis represents the total acres burned by fires each year. The units on the y-axis indicate the scale of the total acres burned, which is the variable being measured. 

In [ ]:
# Group data by 'Fire_Year' and calculate the total acres burned per year
acres_burned_per_year = wildfire_df.groupby('Fire_Year')['GIS_Acres'].sum()

# Create the time series graph
plt.figure(figsize=(10, 6))
acres_burned_per_year.plot(kind='line', marker='o', linestyle='-', color='b')
plt.xlabel('Year')
plt.ylabel('Total Acres Burned')
plt.title('Time Series of Total Acres Burned per Year')
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
#load the AQI data from the file
with open("aqi.json", 'r') as json_file:
    aqi_dict = json.load(json_file)

In [ ]:
aqi = pd.DataFrame.from_dict(aqi_dict, orient='index', columns = ["AQI"]).reset_index()
aqi["Fire_Year"] = aqi["index"]
aqi["Fire_Year"] = aqi['Fire_Year'].astype(int)

In [ ]:
smoke_est = temp.groupby("Fire_Year")["Smoke_Estimate_Scaled"].mean().reset_index()
smoke_est["Fire_Year"] = smoke_est['Fire_Year'].astype(int)

In [ ]:
#merge the two datasets for ease of use for visualizations
merged_df = pd.merge(smoke_est, aqi, on="Fire_Year")

This graph is a time series graph containing your fire smoke estimate and the AQI estimate for Benicia, CA. The x-axis represents the years during which the fires occurred. The Y-axis represents the estimated smoke impact on the city of Benicia. The orange graph represents the AQI. The Air Quality Index is EPA’s index for reporting air quality. The blue graph represents the smoke estimate that I calculated by dividing the area across which the wildfire had spread by the distance from the fire.

In [ ]:
# Create the time series graph

merged_df['Fire_Year'] = pd.to_datetime(merged_df['Fire_Year'])

# Create the time series graph
plt.figure(figsize=(15, 7))
plt.plot(merged_df['Fire_Year'], merged_df['Smoke_Estimate_Scaled'], label='Fire Smoke Estimate', marker='o', linestyle='-')
plt.plot(merged_df['Fire_Year'], merged_df['AQI'], label='AQI Estimate', marker='o', linestyle='-')

# Set labels and title
plt.xlabel('Year')
plt.ylabel('Estimate')
plt.title('Time Series of Fire Smoke Estimate and AQI Estimate')
plt.legend()

# Show the graph
plt.grid(True)
plt.show()